## Imports

In [ ]:
!pip install transformers

!pip install datasets

!pip install sacrebleu

In [ ]:
!pip install torch==1.11.0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

## Test Dataset Loading

In [ ]:
BASE_PATH = "/content/gdrive/MyDrive/experiments"

CLASSIFIER_CHECKPOINT = f"{BASE_PATH}/classifier/checkpoint-30"
UNRESTRICTED_CHECKPOINT = f"{BASE_PATH}/unrestricted_split/checkpoint-8750/"
RESTRICTED_CHECKPOINT = f"{BASE_PATH}/restricted/checkpoint-3125/"

UNRESTRICTED_TEST_PATH = f"{BASE_PATH}/unrestricted_test_dataset.csv"
RESTRICTED_TEST_PATH = f"{BASE_PATH}/restricted_test_dataset.csv"

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

unrestricted_test_set = load_dataset('csv', data_files=UNRESTRICTED_TEST_PATH)
restricted_test_set = load_dataset('csv', data_files=RESTRICTED_TEST_PATH)

unrestricted_test_set['train'] = unrestricted_test_set['train'].add_column('label', [0] * len(unrestricted_test_set['train']))
restricted_test_set['train'] = restricted_test_set['train'].add_column('label', [1] * len(restricted_test_set['train']))

test_dataset = concatenate_datasets([unrestricted_test_set['train'], restricted_test_set['train']]).shuffle()

dataset = DatasetDict({'test': test_dataset})

dataset

## Classifier

In [ ]:
from transformers import AutoTokenizer

cls_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return cls_tokenizer(examples["en"], truncation=True)

In [ ]:
tokenized_dataset = test_dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
from transformers import DataCollatorWithPadding

cls_data_collator = DataCollatorWithPadding(tokenizer=cls_tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

cls_model = AutoModelForSequenceClassification.from_pretrained(CLASSIFIER_CHECKPOINT, num_labels=2)

cls_model.resize_token_embeddings(len(cls_tokenizer))

In [ ]:
print("For T5:")
print("Tokenizer vocab_size: {}".format(cls_tokenizer.vocab_size))
print("Model vocab size: {}\n".format(cls_model.config.vocab_size))

In [ ]:
RESULTS_DIR = "/content/gdrive/MyDrive/experiments/ensemble"

STEPS = 31

cls_training_args = TrainingArguments(
    output_dir=RESULTS_DIR,
    evaluation_strategy="steps",
    eval_steps=STEPS,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    # num_train_epochs=1,
    max_steps=STEPS,
    adam_beta2=0.98,
    warmup_steps=100,
    optim="adamw_torch",
    fp16=True,
    save_strategy='steps',
    save_steps=STEPS-1
)

In [ ]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import precision_recall_fscore_support

accuracy = load_metric("accuracy")

def cls_compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    preds = eval_preds.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy.compute(references=labels, predictions=preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'preds': preds,
        'labels': labels
    }

In [ ]:
cls_trainer = Trainer(
    model=cls_model,
    args=cls_training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=cls_tokenizer,
    data_collator=cls_data_collator,
    compute_metrics=cls_compute_metrics
)

# trainer.train("/content/gdrive/MyDrive/final_thesis_experiments/checkpoint-5625/")
# trainer.train()

In [ ]:
evaluation = cls_trainer.evaluate(tokenized_dataset)

evaluation

In [ ]:
cls_prediction = evaluation['eval_preds']

cls_prediction

## Unrestricted

In [ ]:
unrestricted_dataset = test_dataset.filter(lambda example, idx: cls_prediction[idx] == 0, with_indices=True)

unrestricted_dataset = unrestricted_dataset.remove_columns("label")

unrestricted_dataset

In [ ]:
from transformers import AutoTokenizer

unrestricted_tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
source_lang = "en"
target_lang = "ltl"
prefix = "translate English to LTL: "


def preprocess_function(examples):
    inputs = [prefix + example.replace('_', ' _ ').replace(',', ' ,') for example in examples[source_lang]]
    targets = [example.replace('_', ' _ ') for example in examples[target_lang]]
    model_inputs = unrestricted_tokenizer(inputs, max_length=256, truncation=True)

    with unrestricted_tokenizer.as_target_tokenizer():
        labels = unrestricted_tokenizer(targets, max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
unrestricted_tokenized_dataset = unrestricted_dataset.map(preprocess_function, batched=True)

In [ ]:
unrestricted_tokenized_dataset

In [ ]:
unrestricted_tokenized_dataset['en'][:10]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

unrestricted_model = AutoModelForSeq2SeqLM.from_pretrained(UNRESTRICTED_CHECKPOINT)

unrestricted_model.resize_token_embeddings(len(unrestricted_tokenizer))

In [ ]:
print("For T5:")
print("Tokenizer vocab_size: {}".format(unrestricted_tokenizer.vocab_size))
print("Model vocab size: {}\n".format(unrestricted_model.config.vocab_size))

In [ ]:
from transformers import DataCollatorForSeq2Seq

unrestricted_data_collator = DataCollatorForSeq2Seq(tokenizer=unrestricted_tokenizer, model=unrestricted_model)

In [ ]:
unrestricted_training_args = Seq2SeqTrainingArguments(
    output_dir=RESULTS_DIR,
    evaluation_strategy="epoch",
    learning_rate=0.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    adam_beta2=0.98,
    warmup_steps=4000,
    optim="adamw_torch",
    fp16=True,
    predict_with_generate=True,
    generation_max_length=256,
    save_strategy='epoch'
)

In [ ]:
import numpy as np
from datasets import load_metric

sacrebleu = load_metric("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = unrestricted_tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, unrestricted_tokenizer.pad_token_id)
    decoded_labels = unrestricted_tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != unrestricted_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    result['preds'] = decoded_preds
    result['lbls'] = decoded_labels
    return result

In [ ]:
unrestricted_trainer = Seq2SeqTrainer(
    model=unrestricted_model,
    args=unrestricted_training_args,
    train_dataset=unrestricted_tokenized_dataset,
    eval_dataset=unrestricted_tokenized_dataset,
    tokenizer=unrestricted_tokenizer,
    data_collator=unrestricted_data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train("/content/gdrive/MyDrive/final_thesis_experiments/checkpoint-5625/")
# trainer.train()

In [ ]:
unrestricted_results = unrestricted_trainer.evaluate(unrestricted_tokenized_dataset, max_length=256)

unrestricted_results

In [ ]:
unrestricted_results = {
    'preds': unrestricted_results['eval_preds'],
    'labels': unrestricted_results['eval_lbls']
    }

unrestricted_results

## Domain-Specific

In [ ]:
restricted_dataset = test_dataset.filter(lambda example, idx: cls_prediction[idx] == 1, with_indices=True)

restricted_dataset = restricted_dataset.remove_columns("label")

restricted_dataset

In [ ]:
from transformers import AutoTokenizer

restricted_tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
source_lang = "en"
target_lang = "ltl"
prefix = "translate English to LTL: "


def preprocess_function(examples):
    inputs = [prefix + example.replace(',', ' ,') for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = restricted_tokenizer(inputs, max_length=256, truncation=True)

    with restricted_tokenizer.as_target_tokenizer():
        labels = restricted_tokenizer(targets, max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
restricted_tokenized_dataset = restricted_dataset.map(preprocess_function, batched=True)

In [ ]:
restricted_tokenized_dataset

In [ ]:
restricted_tokenized_dataset['en'][:10]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

restricted_model = AutoModelForSeq2SeqLM.from_pretrained(RESTRICTED_CHECKPOINT)

restricted_model.resize_token_embeddings(len(restricted_tokenizer))

In [ ]:
print("For T5:")
print("Tokenizer vocab_size: {}".format(restricted_tokenizer.vocab_size))
print("Model vocab size: {}\n".format(restricted_model.config.vocab_size))

In [ ]:
from transformers import DataCollatorForSeq2Seq

restricted_data_collator = DataCollatorForSeq2Seq(tokenizer=restricted_tokenizer, model=restricted_model)

In [ ]:
restricted_training_args = Seq2SeqTrainingArguments(
    output_dir=RESULTS_DIR,
    evaluation_strategy="epoch",
    learning_rate=0.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    adam_beta2=0.98,
    warmup_steps=2500,
    optim="adamw_torch",
    fp16=True,
    predict_with_generate=True,
    generation_max_length=256,
    save_strategy='epoch'
)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = restricted_tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, restricted_tokenizer.pad_token_id)
    decoded_labels = restricted_tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != restricted_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    result['preds'] = decoded_preds
    result['lbls'] = decoded_labels
    return result

In [ ]:
restricted_trainer = Seq2SeqTrainer(
    model=restricted_model,
    args=restricted_training_args,
    train_dataset=restricted_tokenized_dataset,
    eval_dataset=restricted_tokenized_dataset,
    tokenizer=restricted_tokenizer,
    data_collator=restricted_data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train("/content/gdrive/MyDrive/final_thesis_experiments/checkpoint-5625/")
# trainer.train()

In [ ]:
restricted_results = restricted_trainer.evaluate(restricted_tokenized_dataset, max_length=256)

restricted_results

In [ ]:
restricted_results = {
    'preds': restricted_results['eval_preds'],
    'labels': restricted_results['eval_lbls']
    }

restricted_results

## Final Results Evaluation

In [ ]:
final_labels = unrestricted_results['labels'] + restricted_results['labels']
final_preds = unrestricted_results['preds'] + restricted_results['preds']

len(final_labels), len(final_preds)

In [ ]:
sacrebleu.compute(predictions=final_preds, references=final_labels)